# Assignment: Sentiment Analysis in Twitter
## SN:20152948

## 1. Import Lab

In [1]:
# Import Lab
import pandas as pd
import numpy as np
import os
import re
import tensorflow as tf

import time
import datetime

from tensorflow.contrib import learn
import codecs

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

# Program Running Display
from tqdm import tqdm

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
# Check Tensorflow-gpu Configuration
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)

# tf.test.is_gpu_available()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7507533217613343321
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4941427507
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10557702081856170601
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
1.14.0


## 1. Data Loading and Pre-processing

In [3]:
### Class 'Data': Loading Data, Cleaning Data and Generate Dictionary
# function data_load
# function data_clean
# function dict_set

class Data:
    def __init__(self, task_type, Glove_flag = True):
        self.df = pd.DataFrame()
        self.task_type = task_type
        self.Glove_flag = Glove_flag
        if self.task_type == 'A':
            self.file_path = os.path.join('datasets', '4A', 'SemEval2017-task4-dev.subtask-A.english.INPUT.txt')
            self.label_list = ['negative', 'neutral', 'positive']
        elif task_type == 'B':     
            self.file_path = os.path.join('datasets', '4B', 'SemEval2017-task4-dev.subtask-BD.english.INPUT.txt')
            self.label_list = ['negative', 'positive']     
        else:
            print('ERROR: Cannot find Task type: ', task_type, '!!!')
            
        # Function below will run automatically when class is set    
        # Load data and save in dataframe
        def data_load(self):        
            list_X = []
            list_Y = []
            list_topic = []

            file = open(self.file_path, 'r')
            if self.task_type == 'A':
                for line in file:
                    if len(line) > 18:# Except for line with only '\t'
                        list_X.append(line.split('\t', 2)[2])
                        list_Y.append(line.split('\t', 2)[1])

            elif self.task_type == 'B':     
                for line in file:
                    if len(line) > 18:
                        list_X.append(line.split('\t', 3)[3])
                        list_Y.append(line.split('\t', 3)[2])
                        list_topic.append(line.split('\t', 3)[1])

            self.df['X'] = list_X
            self.df['Y'] = list_Y
            if self.task_type == 'B':
                self.df['topic'] = list_topic
                # Add topic words into X
                self.df['X'] = self.df['X'] + ' ' + self.df['topic']
            file.close()
            
            

        def data_clean(self):
            # Pre-process of text
            for i in range(self.df.shape[0]):

                # Read each line in X and clean all the '/t/n' at the beginning and end
                line = self.df['X'][i]
                line = line.strip()

                # Emojis Replace
                line = re.sub(r':\)','<emoji_1>',line)
                line = re.sub(r':P','<emoji_2>',line)
                line = re.sub(r':p','<emoji_3>',line)
                line = re.sub(r':>','<emoji_4>',line)
                line = re.sub(r':3','<emoji_5>',line)
                line = re.sub(r':D','<emoji_6>',line)
                line = re.sub(r' XD ','<emoji_7>',line)
                line = re.sub(r' <3 ','<emoji_8>',line)

                line = re.sub(r':\(','<emoji_9>',line)
                line = re.sub(r':<','<emoji_10>',line)
                line = re.sub(r':<','<emoji_11>',line)
                line = re.sub(r'>:\(','<emoji_12>',line)

                # Mention User Replace 
                line = re.sub(r'@(\S*)', '<mention_user>', line)

                # URL Replace 
                line = re.sub(r'http(\S*)', '<url>', line)

    #             # Delete all the quotation marks
    #             line = re.sub('\'', '', line)
    #             line = re.sub('\"', '', line)

                # Clean Unimportant Word and Separate Symbols
                line = re.sub(r'[^A-Za-z0-9(),!?\'\`]', ' ', line)
                line = re.sub(r'\'s', ' \'s', line)
                line = re.sub(r'\'ve', ' \'ve', line)
                line = re.sub(r'n\'t', ' n\'t', line)
                line = re.sub(r'\'re', ' \'re', line)
                line = re.sub(r'\'d', ' \'d', line)
                line = re.sub(r'\'ll', ' \'ll', line)
                line = re.sub(r',', ' , ', line)
                line = re.sub(r'!', ' ! ', line)
                line = re.sub(r'\(', ' \( ', line)
                line = re.sub(r'\)', ' \) ', line)
                line = re.sub(r'\?', ' \? ', line)
                line = re.sub(r'\s{2,}', ' ', line)

                # Replace repetitive letter which more than 2 times
                line = re.sub(r'(.)(\1{1})\1+', r'\1\2', line)

                # Lower the letter
                line = line.lower()
                self.df['X'][i] = line

                # Digitization Y
                label = self.df['Y'][i]
                label = self.label_list.index(label)
                self.df['Y'][i] = label
                
                
        def dict_set(self):     
            
            max_sentence_length = max(len(x.split(' ')) for x in self.df['X'])
            # Not using GloVe
            if self.Glove_flag == False:
                self.V_processor = learn.preprocessing.VocabularyProcessor(max_sentence_length)
                self.X = np.array(list(self.V_processor.fit_transform(self.df['X'])))
                self.Y = to_categorical(self.df['Y'])
            else: 

                GloVe_path_name = os.path.join('glove.6B.50d.txt')

                def load_GloVe(GloVe_path_name):
                    vocab = []
                    vector = []
                    file = open(GloVe_path_name, 'r', encoding = 'utf-8')
                    for i, line in enumerate(tqdm(file)):
                        line = line.strip().split(' ')
                        vocab.append(line[0])
                        vector.append(line[1:])

                    file.close()
                    return vocab, vector

                vocab, vector = load_GloVe(GloVe_path_name)

                vocab_num = len(vocab)
                vector_dim = len(vector[0])
                vector = np.array(vector)

                # Training GloVe 
                W = tf.Variable(tf.constant(0.0, shape=[vocab_num, vector_dim]), trainable=False, name='W')
                GloVe_placeholder = tf.placeholder(tf.float32, [vocab_num, vector_dim])
                GloVe_init = W.assign(GloVe_placeholder)

                GloVe_config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
                GloVe = tf.Session(config=GloVe_config)
                self.W = GloVe.run(GloVe_init, feed_dict={GloVe_placeholder: vector})

                # Set Dictionary
                self.V_processor = learn.preprocessing.VocabularyProcessor(max_sentence_length)
                pretrain = self.V_processor.fit(vocab)
                self.X = np.array(list(self.V_processor.transform(self.df['X'])))
                self.Y = to_categorical(self.df['Y'])
            
        # Run initialize function
        data_load(self)
        data_clean(self)
        dict_set(self)
            
         
        

## 2. Batch Generate

In [4]:
#Generate Random Batches
#Source: https://github.com/dennybritz/cnn-text-classification-tf/blob/master/data_helpers.py
def gen_batch(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((data_size-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

## 3. Model Defination

### 3.1 CNN

In [5]:
class CNN(object):
    def __init__(self, num_feature, num_class, size_vocab, size_embed, size_filter, num_filter, Data, L2_reg_lambda=0.0):
        
        self.input_X = tf.placeholder(tf.int32, [None, num_feature], name = 'input_X')
        self.input_Y = tf.placeholder(tf.float32, [None, num_class], name = 'input_Y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name = 'dropout_keep_prob')
 
        # L2 regularization loss
        L2_loss = tf.constant(0.0)

        
        # Word Embedding
        with tf.name_scope('Embedding'):
            self.W = tf.Variable(tf.random_uniform([size_vocab, size_embed], -1.0, 1.0), name='W')
#             self.W = Data.W
            self.embed_chars = tf.nn.embedding_lookup(self.W, self.input_X)
            self.embed_chars_expanded = tf.expand_dims(self.embed_chars, -1)
   
        # Convolution & Maxpooling
        Pool_list = []
        for i, size_f in enumerate(size_filter):
            with tf.name_scope('conv-maxpool-%s' % size_f):
                # Convolution Layer
                filter_shape = [size_f, size_embed, 1, num_filter]
                
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
                b = tf.Variable(tf.constant(0.1, shape=[num_filter]), name='b')
                Conv = tf.nn.conv2d(
                    self.embed_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name='Conv')
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(Conv, b), name='relu')
                # Maxpooling over the outputs
                Pool = tf.nn.max_pool(
                    h,
                    ksize=[1, num_feature - size_f + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name='Pool')
                Pool_list.append(Pool)
        
        # Merge Result
        num_filter_total = num_filter * len(size_filter)
        self.h_pool = tf.concat(Pool_list, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filter_total])
        
        # Dropout
        with tf.name_scope('Dropout'):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Score and Predictions
        with tf.name_scope('Output'):
            # NN Parameter Set
            W = tf.get_variable(
                'W',
                shape=[num_filter_total, num_class],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_class]), name='b')
            # L2 Loss
            L2_loss += tf.nn.l2_loss(W)
            L2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name='scores')
            self.pred = tf.argmax(self.scores, 1, name='predictions')
            
        # Loss Calculation
        with tf.name_scope('Loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_Y)
            self.loss = tf.reduce_mean(losses) + L2_reg_lambda * L2_loss

            
        # Confusion Matrix
        with tf.name_scope('Confusion_Matrix'):
            M = tf.contrib.metrics.confusion_matrix(tf.argmax(self.input_Y, 1), self.pred)
            acc_list = []
            recall_list = []
            
            row_sum = tf.reduce_sum(M, axis = 1)
            col_sum = tf.reduce_sum(M, axis = 0)
            
            acc_list = tf.div_no_nan(tf.cast(tf.diag_part(M), 'float'), tf.cast(col_sum, 'float'))
            recall_list = tf.div_no_nan(tf.cast(tf.diag_part(M), 'float'), tf.cast(row_sum, 'float'))
          
            
            self.acc = tf.reduce_mean(acc_list)
            self.recall = tf.reduce_mean(recall_list)
            
            temp = tf.multiply(tf.constant(2, dtype = 'float'), tf.multiply(self.acc, self.recall))
            self.F1_score = tf.div(temp, tf.add(self.acc, self.recall))
            


### 3.2 LSTM

In [6]:
class LSTM(object):
    def __init__(self, num_feature, num_class, size_vocab, size_embed, L2_reg_lambda = 0.0, num_hidden = 100):
        
        
        self.input_X = tf.placeholder(tf.int32, [None, num_feature], name = 'input_X')
        self.input_Y = tf.placeholder(tf.float32, [None, num_class], name = 'input_Y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name = 'dropout_keep_prob')
 
        L2_loss = tf.constant(0.0) # Keeping track of L2 regularization loss
    
        # Embedding Layer
        with tf.device('/cpu:0'), tf.name_scope('embedding'):
            self.W = tf.Variable(tf.random_uniform([size_vocab, size_embed], -1.0, 1.0),name='W')
            self.embed_chars = tf.nn.embedding_lookup(self.W, self.input_X)

        # LSTM Layer
        self.lstm_cell = tf.contrib.rnn.LSTMCell(num_hidden,state_is_tuple=True)
        self.lstm_out,self.lstm_state = tf.nn.dynamic_rnn(self.lstm_cell,self.embed_chars,dtype=tf.float32)

        val2 = tf.transpose(self.lstm_out, [1, 0, 2])
        last = tf.gather(val2, int(val2.get_shape()[0]) - 1) 

        out_weight = tf.Variable(tf.random_normal([num_hidden, num_class]))
        out_bias = tf.Variable(tf.random_normal([num_class]))

        with tf.name_scope('Output'):
            self.scores = tf.nn.xw_plus_b(last, out_weight,out_bias, name='scores')
            self.pred = tf.nn.softmax(self.scores, name='predictions')

        with tf.name_scope('Loss'):
            self.losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores,labels=self.input_Y)
            self.loss = tf.reduce_mean(self.losses, name='loss')

        # Confusion Matrix
        with tf.name_scope('Confusion_Matrix'):
            M = tf.contrib.metrics.confusion_matrix(tf.argmax(self.input_Y, 1), tf.argmax(self.pred, 1))
            acc_list = []
            recall_list = []
            
            row_sum = tf.reduce_sum(M, axis = 1)
            col_sum = tf.reduce_sum(M, axis = 0)
            
            acc_list = tf.div_no_nan(tf.cast(tf.diag_part(M), 'float'), tf.cast(col_sum, 'float'))
            recall_list = tf.div_no_nan(tf.cast(tf.diag_part(M), 'float'), tf.cast(row_sum, 'float'))
          
            
            self.acc = tf.reduce_mean(acc_list)
            self.recall = tf.reduce_mean(recall_list)
            
            temp = tf.multiply(tf.constant(2, dtype = 'float'), tf.multiply(self.acc, self.recall))
            self.F1_score = tf.div(temp, tf.add(self.acc, self.recall))     
            
            
            


## 4. Model Training

In [7]:
def model_train(Data, X_Y, model_MODE = 0):
    [X_tr, Y_tr, X_dev, Y_dev, X_tt, Y_tt] = X_Y


    num_checkpoints = 0
    batch_size = 128
    num_epochs = 10 
    dropout_prob = 0.5 
    evaluate_step = 100 
    checkpoint_step = 100000 

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
        session_conf.gpu_options.allow_growth = True
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            if model_MODE == 0:
                model = CNN(X_tr.shape[1], Y_tr.shape[1], len(Data.V_processor.vocabulary_), 32, [3,4,5], 32, Data, 0.0)
            elif model_MODE == 1:
                model = LSTM(X_tr.shape[1], Y_tr.shape[1], len(Data.V_processor.vocabulary_), 32)
            else:
                print('MODE ERROR!')

        # Set Optimizer and Update Gradient
        global_step = tf.Variable(0, name = 'Global_Step', trainable = False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(model.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep Track of Gradient Values and Sparsity
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram('{}/grad/hist'.format(v.name), g)
                sparsity_summary = tf.summary.scalar('{}/grad/sparsity'.format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output Directory for Models and Summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, 'runs', timestamp))
        print('Writing to {}\n'.format(out_dir))

         # Summaries for Loss and Accuracy
        loss_summary = tf.summary.scalar('loss', model.loss)
        recall_summary = tf.summary.scalar('recall', model.recall)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, recall_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, 'summaries', 'train')
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev Summaries
        dev_summary_op = tf.summary.merge([loss_summary, recall_summary])
        dev_summary_dir = os.path.join(out_dir, 'summaries', 'dev')
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint Directory.
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, 'checkpoints'))
        checkpoint_prefix = os.path.join(checkpoint_dir, 'model')
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write Vocabulary Dictionary
        Data.V_processor.save(os.path.join(out_dir, 'vocab'))

        # Initialize All Variables
        sess.run(tf.global_variables_initializer())

        # Train function
        def train_step(X_batch, Y_batch,save=False):
            feed_dict = {
              model.input_X: X_batch,
              model.input_Y: Y_batch,
              model.dropout_keep_prob: dropout_prob
            }
            _, step, summaries, loss, recall = sess.run(
                [train_op, global_step, train_summary_op, model.loss, model.recall],
                feed_dict)
            print('---> step {}, loss {:g}, recall {:g}'.format(step, loss, recall))
            if save:
                train_summary_writer.add_summary(summaries, step)

        # Evaluate function
        def dev_step(x_batch, y_batch, writer=None,save=False):
            feed_dict = {
              model.input_X: X_batch,
              model.input_Y: Y_batch,
              model.dropout_keep_prob: 0.5
            }
            step, summaries, loss, recall, acc, F1_score = sess.run(
                [global_step, dev_summary_op, model.loss, model.recall, model.acc, model.F1_score],
                feed_dict)
            print('---> step {}, loss {:g}, recall {:g}, acc {:g}, F1 score {:g}'.format(step, loss, recall, acc, F1_score))

            if save:
                if writer:
                    writer.add_summary(summaries, step)
                    
            return [recall, acc, F1_score]
            
        # Generate Batch
        batches = gen_batch(list(zip(X_tr, Y_tr)), batch_size, num_epochs)

        # Train in Batch
        for batch in batches:
            X_batch, Y_batch = zip(*batch)
            train_step(X_batch, Y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_step == 0:
                print('\nEvaluation:')
                _ = dev_step(X_dev, Y_dev, writer=dev_summary_writer)
                print('')
            if current_step % checkpoint_step == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print('Saved model checkpoint to {}\n'.format(path))
        
        # Evaluate for each batch
        print('\nTraining Result:')
        test_result_list = []
        test_result_list = dev_step(X_tt, Y_tt, writer=dev_summary_writer)

        return test_result_list



## 5. Dataset and Subtask Split 

In [8]:
# Split function
def data_split(X, Y):


    X_re, X_tt, Y_re, Y_tt = train_test_split(X, Y,
                                             test_size = 0.3,
#                                              stratify = Y,
                                             random_state = 5)

    X_tr, X_dev, Y_tr, Y_dev = train_test_split(X_re, Y_re,
                                             test_size = 0.1,
#                                              stratify = Y_re,
                                             random_state = 5)

    return [X_tr, Y_tr, X_dev, Y_dev, X_tt, Y_tt]

def senti_analy(Data, model_MODE, change_flag = False):
    
    X_total = []
    Y_total = []
    X_Y = []
   
    
    if Data.task_type == 'A' or change_flag == True:
        X_total = Data.X
        Y_total = Data.Y
        
        X_Y = data_split(X_total, Y_total)
        [recall, acc, F1_score] = model_train(Data, X_Y, model_MODE)

        
    elif Data.task_type == 'B':
        topic_list = Data.df['topic'].unique()
        for topic in topic_list:
            X_total.append(Data.X[Data.df['topic'] == topic])
            Y_total.append(Data.Y[Data.df['topic'] == topic])    
    
        recall_list = []
        acc_list = []
        F1_score_list = []
        for i, _ in enumerate(X_total):
            print('--->Topic {}: Now training for topic \'{}\''.format(i, topic_list[i]))
            X_Y = data_split(X_total[i], Y_total[i])
            [recall, acc, F1_score] = model_train(Data, X_Y, model_MODE)
            recall_list.append(recall)
            acc_list.append(acc)
            F1_score_list.append(F1_score)
        recall = np.mean(recall_list)
        acc = np.mean(acc_list)
        F1_score = np.mean(F1_score_list)
        
        
    else:
        print('ERROR: Cannot find Task type: ', Data.task_type, '!!!')
            

    print('For task {}, recall is {:g}, accuracy is {:g}, F1 score is {:g}'.format(Data.task_type, recall, acc, F1_score))          
            
    
    




## 6. Program Running

### 6.1 Generate Datasets

In [9]:
A_dict = Data('A', Glove_flag=False)
A_GloVe = Data('A', Glove_flag=True)
B_dict = Data('B', Glove_flag=False)
B_GloVe = Data('B', Glove_flag=True)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


400000it [00:03, 111012.06it/s]
400000it [00:03, 117710.90it/s]


### 6.2 Subtask A

#### 6.2.1 Subtask A: Normal Dictionary with CNN

In [10]:
senti_analy(A_dict, model_MODE = 0)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/gr

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 2.73856, recall 0.348473
---> step 2, loss 2.79866, recall 0.358549
---> step 3, loss 2.61769, recall 0.318878
---> step 4, loss 2.14811, recall 0.381169
---> step 5, loss 2.02945, recall 0.375196
---> step 6, loss 2.15144, recall 0.283981
---> step 7, loss 1.79728, recall 0.340537
---> step 8, loss 1.53857, recall 0.384297
---> step 9, loss 1.94561, recall 0.299387
---> step 10, loss 1.62818, recall 0.384563
---> step 11, loss 1.50214, recall 0.352455
---> step 12, loss 1.58259, recall 0.4579
---> step 13, loss 1.79747, recall 0.283333
---> step 14, loss 1.49876, recall 0.398746
---> step 15, loss 1.64331, recall 0.342229
---> step 16, loss 1.73741, recall 0.272611
---> step 17, loss 1.68394, recall 0.313688
---> step 18, loss 1.54062, recall 0.338934
---> step 19, loss 1.96331, recall 0.282972
---> step 20, loss 1.8259, recall 0.287422
---> step 21, loss 1.33702, recall 0.356434
---> step 22, loss 1.63828, recall 0.283518
---> step 23, loss 1.62842, recall 0.370242


---> step 185, loss 1.13108, recall 0.393993
---> step 186, loss 1.14924, recall 0.336745
---> step 187, loss 1.30433, recall 0.350567
---> step 188, loss 1.08429, recall 0.392135
---> step 189, loss 1.18807, recall 0.351634
---> step 190, loss 1.14506, recall 0.36217
---> step 191, loss 1.19504, recall 0.379
---> step 192, loss 1.16283, recall 0.343533
---> step 193, loss 1.20644, recall 0.379537
---> step 194, loss 1.11587, recall 0.338818
---> step 195, loss 1.10736, recall 0.408386
---> step 196, loss 1.16048, recall 0.38359
---> step 197, loss 1.10918, recall 0.364595
---> step 198, loss 1.18522, recall 0.368035
---> step 199, loss 1.15056, recall 0.388242
---> step 200, loss 1.16446, recall 0.332769

Evaluation:
---> step 200, loss 1.12268, recall 0.377508, acc 0.369369, F1 score 0.373394

---> step 201, loss 1.27025, recall 0.326929
---> step 202, loss 1.1119, recall 0.398609
---> step 203, loss 1.10068, recall 0.349879
---> step 204, loss 1.22067, recall 0.33462
---> step 205, 

---> step 369, loss 0.998063, recall 0.37284
---> step 370, loss 0.939397, recall 0.430327
---> step 371, loss 1.04924, recall 0.396217
---> step 372, loss 0.995034, recall 0.422999
---> step 373, loss 1.01285, recall 0.394393
---> step 374, loss 1.04181, recall 0.327755
---> step 375, loss 1.0534, recall 0.36438
---> step 376, loss 1.11423, recall 0.316473
---> step 377, loss 1.02275, recall 0.382586
---> step 378, loss 0.977374, recall 0.345805
---> step 379, loss 1.05288, recall 0.394532
---> step 380, loss 0.970038, recall 0.348189
---> step 381, loss 1.06477, recall 0.386585
---> step 382, loss 1.0243, recall 0.386691
---> step 383, loss 1.01203, recall 0.351573
---> step 384, loss 1.02576, recall 0.428912
---> step 385, loss 1.00194, recall 0.451697
---> step 386, loss 1.0689, recall 0.351252
---> step 387, loss 0.961807, recall 0.478324
---> step 388, loss 1.1143, recall 0.332887
---> step 389, loss 0.97434, recall 0.40793
---> step 390, loss 1.01059, recall 0.412239
---> step 3

---> step 548, loss 0.90571, recall 0.437757
---> step 549, loss 0.904686, recall 0.399493
---> step 550, loss 0.94844, recall 0.437773
---> step 551, loss 0.826986, recall 0.417677
---> step 552, loss 1.01613, recall 0.413612
---> step 553, loss 0.939632, recall 0.416494
---> step 554, loss 0.936036, recall 0.41296
---> step 555, loss 0.948854, recall 0.406298
---> step 556, loss 1.03563, recall 0.363385
---> step 557, loss 0.930363, recall 0.400852
---> step 558, loss 0.92218, recall 0.365041
---> step 559, loss 0.956545, recall 0.495144
---> step 560, loss 0.897287, recall 0.445397
---> step 561, loss 0.910848, recall 0.436941
---> step 562, loss 0.880076, recall 0.393275
---> step 563, loss 0.983145, recall 0.436512
---> step 564, loss 0.999542, recall 0.445129
---> step 565, loss 0.952417, recall 0.496197
---> step 566, loss 0.939576, recall 0.457138
---> step 567, loss 1.03819, recall 0.410684
---> step 568, loss 0.977621, recall 0.470167
---> step 569, loss 0.926078, recall 0.48

---> step 730, loss 0.851453, recall 0.50694
---> step 731, loss 0.864455, recall 0.501736
---> step 732, loss 0.95295, recall 0.463886
---> step 733, loss 0.996602, recall 0.444444
---> step 734, loss 0.952525, recall 0.4125
---> step 735, loss 0.931107, recall 0.401494
---> step 736, loss 0.910772, recall 0.444755
---> step 737, loss 0.846348, recall 0.464675
---> step 738, loss 0.94916, recall 0.440257
---> step 739, loss 1.01865, recall 0.40892
---> step 740, loss 0.881829, recall 0.533565
---> step 741, loss 1.00484, recall 0.394808
---> step 742, loss 0.941206, recall 0.453424
---> step 743, loss 0.862994, recall 0.443277
---> step 744, loss 0.954512, recall 0.437225
---> step 745, loss 0.993293, recall 0.348161
---> step 746, loss 0.928332, recall 0.485605
---> step 747, loss 0.895569, recall 0.440693
---> step 748, loss 0.859052, recall 0.496778
---> step 749, loss 0.90906, recall 0.39946
---> step 750, loss 0.94051, recall 0.429822
---> step 751, loss 0.949146, recall 0.43874


---> step 907, loss 0.930841, recall 0.481235
---> step 908, loss 0.77958, recall 0.556015
---> step 909, loss 0.927223, recall 0.430505
---> step 910, loss 0.88171, recall 0.486456
---> step 911, loss 0.871065, recall 0.481665
---> step 912, loss 0.868272, recall 0.512997
---> step 913, loss 0.832632, recall 0.5063
---> step 914, loss 0.875839, recall 0.443191
---> step 915, loss 0.942447, recall 0.419164
---> step 916, loss 0.869118, recall 0.458333
---> step 917, loss 0.821194, recall 0.544228
---> step 918, loss 0.89332, recall 0.476201
---> step 919, loss 0.965935, recall 0.480594
---> step 920, loss 0.813646, recall 0.526226
---> step 921, loss 0.892314, recall 0.468136
---> step 922, loss 0.79251, recall 0.551587
---> step 923, loss 0.804746, recall 0.510843
---> step 924, loss 0.789882, recall 0.50787
---> step 925, loss 0.80404, recall 0.513015
---> step 926, loss 0.879782, recall 0.51121
---> step 927, loss 0.801564, recall 0.539805
---> step 928, loss 0.817356, recall 0.5524

#### 6.2.2 Subtask A: Normal Dictionary with LSTM

In [11]:
senti_analy(A_dict, model_MODE = 1)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 2.96711, recall 0.333333
---> step 2, loss 1.53206, recall 0.333333
---> step 3, loss 1.16146, recall 0.333333
---> step 4, loss 1.47911, recall 0.333333
---> step 5, loss 1.35639, recall 0.299487
---> step 6, loss 1.1645, recall 0.314819
---> step 7, loss 1.05528, recall 0.333333
---> step 8, loss 1.02761, recall 0.333333
---> step 9, loss 1.03084, recall 0.333333
---> step 10, loss 1.16897, recall 0.333333
---> step 11, loss 1.04685, recall 0.333333
---> step 12, loss 0.971235, recall 0.333333
---> step 13, loss 1.06635, recall 0.333333
---> step 14, loss 1.04945, recall 0.333333
---> step 15, loss 1.19631, recall 0.333333
---> step 16, loss 1.06019, recall 0.333333
---> step 17, loss 0.990366, recall 0.333333
---> step 18, loss 0.99316, recall 0.333333
---> step 19, loss 1.06453, recall 0.333333
---> step 20, loss 1.06332, recall 0.333333
---> step 21, loss 1.04404, recall 0.333333
---> step 22, loss 1.05532, recall 0.333333
---> step 23, loss 1.00918, recall 0.333

---> step 184, loss 1.04363, recall 0.333333
---> step 185, loss 1.01904, recall 0.333333
---> step 186, loss 1.05049, recall 0.333333
---> step 187, loss 0.999138, recall 0.333333
---> step 188, loss 1.02447, recall 0.333333
---> step 189, loss 1.02088, recall 0.333333
---> step 190, loss 1.01524, recall 0.333333
---> step 191, loss 1.04945, recall 0.333333
---> step 192, loss 1.04247, recall 0.333333
---> step 193, loss 1.0246, recall 0.333333
---> step 194, loss 1.01586, recall 0.333333
---> step 195, loss 0.999324, recall 0.338097
---> step 196, loss 1.07234, recall 0.338983
---> step 197, loss 0.999363, recall 0.337142
---> step 198, loss 1.01033, recall 0.333333
---> step 199, loss 0.954271, recall 0.333333
---> step 200, loss 0.954393, recall 0.333333

Evaluation:
---> step 200, loss 0.964109, recall 0.333333, acc 0.177083, F1 score 0.231293

---> step 201, loss 0.975586, recall 0.333333
---> step 202, loss 1.0622, recall 0.333333
---> step 203, loss 1.11548, recall 0.333333
---

---> step 361, loss 0.886351, recall 0.625585
---> step 362, loss 0.99532, recall 0.484823
---> step 363, loss 0.844163, recall 0.467107
---> step 364, loss 0.877796, recall 0.453821
---> step 365, loss 0.955609, recall 0.378202
---> step 366, loss 0.829322, recall 0.341508
---> step 367, loss 0.96546, recall 0.365801
---> step 368, loss 0.873599, recall 0.346644
---> step 369, loss 0.877231, recall 0.376771
---> step 370, loss 0.883226, recall 0.422073
---> step 371, loss 0.843477, recall 0.472298
---> step 372, loss 0.989434, recall 0.505257
---> step 373, loss 0.958452, recall 0.507737
---> step 374, loss 0.880697, recall 0.651558
---> step 375, loss 0.971095, recall 0.557553
---> step 376, loss 0.935675, recall 0.516436
---> step 377, loss 0.915008, recall 0.469214
---> step 378, loss 0.808285, recall 0.383584
---> step 379, loss 0.978133, recall 0.388384
---> step 380, loss 0.898331, recall 0.41156
---> step 381, loss 0.754068, recall 0.417605
---> step 382, loss 0.859982, recall 

---> step 539, loss 0.749769, recall 0.624747
---> step 540, loss 0.745588, recall 0.705755
---> step 541, loss 0.774774, recall 0.599208
---> step 542, loss 0.809791, recall 0.60343
---> step 543, loss 0.731743, recall 0.63591
---> step 544, loss 0.644311, recall 0.718254
---> step 545, loss 0.706744, recall 0.75496
---> step 546, loss 0.754186, recall 0.686905
---> step 547, loss 0.811973, recall 0.586898
---> step 548, loss 0.711208, recall 0.677417
---> step 549, loss 0.679413, recall 0.719698
---> step 550, loss 0.783778, recall 0.600483
---> step 551, loss 0.806744, recall 0.543518
---> step 552, loss 0.732351, recall 0.520083
---> step 553, loss 0.615956, recall 0.548378
---> step 554, loss 0.665475, recall 0.633681
---> step 555, loss 0.729833, recall 0.623027
---> step 556, loss 0.68844, recall 0.631749
---> step 557, loss 0.706778, recall 0.680684
---> step 558, loss 0.742253, recall 0.650289
---> step 559, loss 0.809524, recall 0.63414
---> step 560, loss 0.676305, recall 0.

---> step 717, loss 0.579607, recall 0.694539
---> step 718, loss 0.552413, recall 0.63815
---> step 719, loss 0.592088, recall 0.670971
---> step 720, loss 0.573183, recall 0.679167
---> step 721, loss 0.56867, recall 0.707853
---> step 722, loss 0.54782, recall 0.758778
---> step 723, loss 0.495186, recall 0.793056
---> step 724, loss 0.639903, recall 0.732061
---> step 725, loss 0.576124, recall 0.780596
---> step 726, loss 0.703571, recall 0.779892
---> step 727, loss 0.530507, recall 0.797918
---> step 728, loss 0.630372, recall 0.675682
---> step 729, loss 0.587747, recall 0.710262
---> step 730, loss 0.651425, recall 0.643398
---> step 731, loss 0.509495, recall 0.691406
---> step 732, loss 0.628327, recall 0.68142
---> step 733, loss 0.587172, recall 0.727633
---> step 734, loss 0.687605, recall 0.727269
---> step 735, loss 0.587587, recall 0.790458
---> step 736, loss 0.655341, recall 0.768687
---> step 737, loss 0.52748, recall 0.774731
---> step 738, loss 0.551972, recall 0.

---> step 895, loss 0.545403, recall 0.826995
---> step 896, loss 0.542051, recall 0.769661
---> step 897, loss 0.478915, recall 0.807631
---> step 898, loss 0.528693, recall 0.766651
---> step 899, loss 0.49832, recall 0.785207
---> step 900, loss 0.460503, recall 0.828388

Evaluation:
---> step 900, loss 0.457755, recall 0.828388, acc 0.905365, F1 score 0.865168

---> step 901, loss 0.56635, recall 0.679972
---> step 902, loss 0.513643, recall 0.755866
---> step 903, loss 0.510291, recall 0.73663
---> step 904, loss 0.616711, recall 0.730556
---> step 905, loss 0.456882, recall 0.75745
---> step 906, loss 0.492026, recall 0.822832
---> step 907, loss 0.45343, recall 0.70831
---> step 908, loss 0.577952, recall 0.742622
---> step 909, loss 0.512662, recall 0.791329
---> step 910, loss 0.431232, recall 0.820224
---> step 911, loss 0.511241, recall 0.775988
---> step 912, loss 0.512673, recall 0.720518
---> step 913, loss 0.523182, recall 0.777357
---> step 914, loss 0.589959, recall 0.

#### 6.2.3 Subtask A: GloVe Dictionary with CNN

In [12]:
senti_analy(A_GloVe, model_MODE = 0)

INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 2.06224, recall 0.370251
---> step 2, loss 1.89325, recall 0.320076
---> step 3, loss 2.06775, recall 0.379406
---> step 4, loss 2.00982, recall 0.338024
---> step 5, loss 1.86396, recall 0.335839
---> step 6, loss 1.92482, recall 0.308426
---> step 7, loss 1.80937, recall 0.276709
---> step 8, loss 1.62193, recall 0.318065
---> step 9, loss 1.61028, recall 0.35
---> step 10, loss 1.39654, recall 0.445035
---> step 11, loss 1.78122, recall 0.262732
---> step 12, loss 1.59738, recall 0.32558
---> step 13, loss 1.52935, recall 0.352473
---> step 14, loss 1.70815, recall 0.288155
---> step 15, loss 1.81784, recall 0.29869
---> step 16, loss 1.68848, recall 0.305481
---> step 17, loss 1.98134, recall 0.321006
---> step 18, loss 1.50583, recall 0.340173
---> step 19, loss 1.5289, recall 0.372687
---> step 20, loss 1.76881, recall 0.328014
---> step 21, loss 1.7049, recall 0.366598
---> step 22, loss 1.90224, recall 0.298713
---> step 23, loss 1.70043, recall 0.347677
---> 

---> step 185, loss 1.04682, recall 0.417516
---> step 186, loss 1.12426, recall 0.35172
---> step 187, loss 1.18303, recall 0.371324
---> step 188, loss 1.27562, recall 0.343415
---> step 189, loss 1.1581, recall 0.397483
---> step 190, loss 1.14036, recall 0.358083
---> step 191, loss 1.2614, recall 0.369419
---> step 192, loss 1.06817, recall 0.427397
---> step 193, loss 1.14249, recall 0.307566
---> step 194, loss 1.10592, recall 0.392222
---> step 195, loss 1.15752, recall 0.324804
---> step 196, loss 1.07383, recall 0.354261
---> step 197, loss 1.10531, recall 0.381296
---> step 198, loss 1.10161, recall 0.355383
---> step 199, loss 1.1967, recall 0.356244
---> step 200, loss 1.1509, recall 0.382009

Evaluation:
---> step 200, loss 1.07702, recall 0.414391, acc 0.408585, F1 score 0.411468

---> step 201, loss 1.11698, recall 0.308874
---> step 202, loss 1.1911, recall 0.334742
---> step 203, loss 1.1119, recall 0.354404
---> step 204, loss 1.23839, recall 0.280338
---> step 205, 

---> step 363, loss 1.02211, recall 0.39508
---> step 364, loss 1.02014, recall 0.405342
---> step 365, loss 1.09865, recall 0.379172
---> step 366, loss 0.987591, recall 0.360594
---> step 367, loss 0.985666, recall 0.412305
---> step 368, loss 0.975243, recall 0.446424
---> step 369, loss 1.06842, recall 0.38081
---> step 370, loss 1.03083, recall 0.351511
---> step 371, loss 0.977005, recall 0.474097
---> step 372, loss 1.03157, recall 0.401863
---> step 373, loss 0.939717, recall 0.405093
---> step 374, loss 1.03613, recall 0.348064
---> step 375, loss 0.972639, recall 0.42731
---> step 376, loss 1.00484, recall 0.388961
---> step 377, loss 1.03989, recall 0.41096
---> step 378, loss 1.03611, recall 0.392869
---> step 379, loss 0.865071, recall 0.437554
---> step 380, loss 0.994995, recall 0.389505
---> step 381, loss 0.947848, recall 0.396317
---> step 382, loss 0.968751, recall 0.392847
---> step 383, loss 0.963916, recall 0.406411
---> step 384, loss 0.937323, recall 0.387308
--

---> step 540, loss 0.893049, recall 0.432099
---> step 541, loss 0.931255, recall 0.395162
---> step 542, loss 0.973357, recall 0.399856
---> step 543, loss 0.973834, recall 0.380993
---> step 544, loss 0.982362, recall 0.432828
---> step 545, loss 0.931458, recall 0.454635
---> step 546, loss 1.00241, recall 0.405036
---> step 547, loss 0.969622, recall 0.458603
---> step 548, loss 1.04332, recall 0.398742
---> step 549, loss 0.936406, recall 0.473545
---> step 550, loss 0.938633, recall 0.422749
---> step 551, loss 0.968588, recall 0.394459
---> step 552, loss 0.971433, recall 0.427125
---> step 553, loss 0.878211, recall 0.452625
---> step 554, loss 0.996493, recall 0.404589
---> step 555, loss 0.93831, recall 0.40165
---> step 556, loss 0.938363, recall 0.459235
---> step 557, loss 0.967669, recall 0.418618
---> step 558, loss 0.920168, recall 0.46751
---> step 559, loss 0.939931, recall 0.394743
---> step 560, loss 0.937072, recall 0.489969
---> step 561, loss 0.970945, recall 0.

---> step 716, loss 0.821293, recall 0.551568
---> step 717, loss 0.835664, recall 0.519115
---> step 718, loss 0.915862, recall 0.478372
---> step 719, loss 0.978937, recall 0.461039
---> step 720, loss 0.916611, recall 0.462191
---> step 721, loss 0.919911, recall 0.52874
---> step 722, loss 0.841601, recall 0.487446
---> step 723, loss 0.925947, recall 0.446156
---> step 724, loss 0.920026, recall 0.454261
---> step 725, loss 0.877007, recall 0.466578
---> step 726, loss 0.905894, recall 0.452941
---> step 727, loss 0.878417, recall 0.480769
---> step 728, loss 0.961154, recall 0.398625
---> step 729, loss 0.879406, recall 0.544215
---> step 730, loss 0.935604, recall 0.386672
---> step 731, loss 0.88487, recall 0.483272
---> step 732, loss 0.949179, recall 0.382769
---> step 733, loss 0.920744, recall 0.449851
---> step 734, loss 0.849415, recall 0.556217
---> step 735, loss 0.810495, recall 0.565486
---> step 736, loss 0.886393, recall 0.467882
---> step 737, loss 0.881724, recall

---> step 893, loss 0.822636, recall 0.552411
---> step 894, loss 0.849044, recall 0.519725
---> step 895, loss 0.799256, recall 0.520986
---> step 896, loss 0.894341, recall 0.443862
---> step 897, loss 0.888339, recall 0.516057
---> step 898, loss 0.880571, recall 0.457739
---> step 899, loss 0.857982, recall 0.502856
---> step 900, loss 0.903911, recall 0.491876

Evaluation:
---> step 900, loss 0.820307, recall 0.455103, acc 0.705535, F1 score 0.553301

---> step 901, loss 0.906429, recall 0.470458
---> step 902, loss 0.746901, recall 0.540533
---> step 903, loss 0.963769, recall 0.391864
---> step 904, loss 0.777235, recall 0.50028
---> step 905, loss 0.85046, recall 0.502667
---> step 906, loss 0.884277, recall 0.472222
---> step 907, loss 0.933454, recall 0.451389
---> step 908, loss 0.896317, recall 0.415612
---> step 909, loss 0.839802, recall 0.520764
---> step 910, loss 0.891389, recall 0.44267
---> step 911, loss 0.843609, recall 0.496276
---> step 912, loss 0.879797, recall

#### 6.2.4 Subtask A: GloVe Dictionary with LSTM

In [13]:
senti_analy(A_GloVe, model_MODE = 1)

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 2.99637, recall 0.333333
---> step 2, loss 1.76333, recall 0.333333
---> step 3, loss 1.16485, recall 0.333333
---> step 4, loss 1.41001, recall 0.333333
---> step 5, loss 1.56806, recall 0.333333
---> step 6, loss 1.75451, recall 0.333333
---> step 7, loss 1.49921, recall 0.333333
---> step 8, loss 1.45303, recall 0.277466
---> step 9, loss 1.20337, recall 0.333333
---> step 10, loss 1.0415, recall 0.333333
---> step 11, loss 1.0365, recall 0.333333
---> step 12, loss 1.13639, recall 0.333333
---> step 13, loss 1.11541, recall 0.333333
---> step 14, loss 1.06295, recall 0.333333
---> step 15, loss 1.1148, recall 0.333333
---> step 16, loss 1.1246, recall 0.343915
---> step 17, loss 1.14112, recall 0.333333
---> step 18, loss 1.12851, recall 0.347826
---> step 19, loss 1.09324, recall 0.333333
---> step 20, loss 1.075, recall 0.333333
---> step 21, loss 1.04489, recall 0.343393
---> step 22, loss 1.05084, recall 0.333333
---> step 23, loss 1.02005, recall 0.333333
---

---> step 183, loss 0.985301, recall 0.333333
---> step 184, loss 1.00034, recall 0.333333
---> step 185, loss 1.03745, recall 0.333333
---> step 186, loss 1.03102, recall 0.333333
---> step 187, loss 0.956867, recall 0.333333
---> step 188, loss 1.00498, recall 0.333333
---> step 189, loss 1.0318, recall 0.333333
---> step 190, loss 0.996077, recall 0.333333
---> step 191, loss 1.06205, recall 0.333333
---> step 192, loss 0.959358, recall 0.333333
---> step 193, loss 1.01079, recall 0.333333
---> step 194, loss 0.971589, recall 0.333333
---> step 195, loss 1.00682, recall 0.333333
---> step 196, loss 1.03171, recall 0.333333
---> step 197, loss 1.03991, recall 0.333333
---> step 198, loss 1.03367, recall 0.333333
---> step 199, loss 1.03983, recall 0.333333
---> step 200, loss 1.03164, recall 0.333333

Evaluation:
---> step 200, loss 1.02807, recall 0.333333, acc 0.158854, F1 score 0.215168

---> step 201, loss 1.00305, recall 0.333333
---> step 202, loss 0.992584, recall 0.333333
---

---> step 359, loss 0.916975, recall 0.479342
---> step 360, loss 0.792399, recall 0.460395
---> step 361, loss 0.932402, recall 0.471214
---> step 362, loss 0.898656, recall 0.434028
---> step 363, loss 0.832734, recall 0.549466
---> step 364, loss 0.879076, recall 0.588203
---> step 365, loss 0.82383, recall 0.624218
---> step 366, loss 0.870271, recall 0.505495
---> step 367, loss 0.846576, recall 0.584091
---> step 368, loss 0.868472, recall 0.506614
---> step 369, loss 0.890803, recall 0.493803
---> step 370, loss 0.870027, recall 0.489683
---> step 371, loss 0.84833, recall 0.387781
---> step 372, loss 0.995988, recall 0.368077
---> step 373, loss 0.843042, recall 0.414483
---> step 374, loss 0.935058, recall 0.410086
---> step 375, loss 0.825709, recall 0.468254
---> step 376, loss 0.902713, recall 0.481332
---> step 377, loss 0.915531, recall 0.484629
---> step 378, loss 0.777615, recall 0.611634
---> step 379, loss 0.881517, recall 0.506592
---> step 380, loss 0.925026, recall

---> step 535, loss 0.7933, recall 0.594935
---> step 536, loss 0.750341, recall 0.683123
---> step 537, loss 0.756759, recall 0.633239
---> step 538, loss 0.671916, recall 0.705226
---> step 539, loss 0.616715, recall 0.713322
---> step 540, loss 0.65608, recall 0.655137
---> step 541, loss 0.770423, recall 0.618524
---> step 542, loss 0.688118, recall 0.657593
---> step 543, loss 0.622824, recall 0.674342
---> step 544, loss 0.648999, recall 0.639257
---> step 545, loss 0.754212, recall 0.519256
---> step 546, loss 0.630717, recall 0.700244
---> step 547, loss 0.860441, recall 0.593
---> step 548, loss 0.695387, recall 0.613947
---> step 549, loss 0.760934, recall 0.63542
---> step 550, loss 0.63464, recall 0.710734
---> step 551, loss 0.701143, recall 0.587541
---> step 552, loss 0.614067, recall 0.600171
---> step 553, loss 0.675326, recall 0.507993
---> step 554, loss 0.779186, recall 0.548609
---> step 555, loss 0.744789, recall 0.559682
---> step 556, loss 0.816136, recall 0.478

---> step 710, loss 0.638061, recall 0.663272
---> step 711, loss 0.687183, recall 0.676386
---> step 712, loss 0.601764, recall 0.725379
---> step 713, loss 0.600805, recall 0.732292
---> step 714, loss 0.609348, recall 0.698413
---> step 715, loss 0.675154, recall 0.646952
---> step 716, loss 0.553358, recall 0.694081
---> step 717, loss 0.515337, recall 0.768552
---> step 718, loss 0.480109, recall 0.785659
---> step 719, loss 0.587809, recall 0.734308
---> step 720, loss 0.603738, recall 0.704476
---> step 721, loss 0.526469, recall 0.748296
---> step 722, loss 0.543657, recall 0.696661
---> step 723, loss 0.547525, recall 0.689825
---> step 724, loss 0.529424, recall 0.726148
---> step 725, loss 0.578774, recall 0.669841
---> step 726, loss 0.616572, recall 0.680279
---> step 727, loss 0.527434, recall 0.750961
---> step 728, loss 0.623886, recall 0.711448
---> step 729, loss 0.614991, recall 0.734021
---> step 730, loss 0.589316, recall 0.79711
---> step 731, loss 0.60483, recall

---> step 887, loss 0.587807, recall 0.694042
---> step 888, loss 0.56612, recall 0.755318
---> step 889, loss 0.546383, recall 0.680086
---> step 890, loss 0.465727, recall 0.820513
---> step 891, loss 0.46477, recall 0.801574
---> step 892, loss 0.464004, recall 0.777284
---> step 893, loss 0.582167, recall 0.762037
---> step 894, loss 0.546798, recall 0.734432
---> step 895, loss 0.489957, recall 0.815736
---> step 896, loss 0.536489, recall 0.729769
---> step 897, loss 0.508058, recall 0.774489
---> step 898, loss 0.503402, recall 0.831904
---> step 899, loss 0.545255, recall 0.778413
---> step 900, loss 0.619547, recall 0.700046

Evaluation:
---> step 900, loss 0.583891, recall 0.718183, acc 0.773492, F1 score 0.744812

---> step 901, loss 0.426759, recall 0.768877
---> step 902, loss 0.419183, recall 0.864662
---> step 903, loss 0.634103, recall 0.721759
---> step 904, loss 0.463239, recall 0.780516
---> step 905, loss 0.49137, recall 0.728185
---> step 906, loss 0.539989, recall

### 6.3 Subtask B: Not Split by Topic

#### 6.3.1 Subtask B: Normal Dictionary with CNN (Not Split)

In [14]:
senti_analy(B_dict, model_MODE = 0, change_flag = True)

INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 2.11214, recall 0.558333
---> step 2, loss 1.84346, recall 0.507164
---> step 3, loss 1.51482, recall 0.490188
---> step 4, loss 1.3193, recall 0.584466
---> step 5, loss 1.32586, recall 0.480538
---> step 6, loss 1.10484, recall 0.44721
---> step 7, loss 1.08839, recall 0.520833
---> step 8, loss 0.975837, recall 0.530129
---> step 9, loss 0.713491, recall 0.575714
---> step 10, loss 0.971115, recall 0.484078
---> step 11, loss 0.794587, recall 0.460455
---> step 12, loss 0.957549, recall 0.455128
---> step 13, loss 0.843801, recall 0.431263
---> step 14, loss 0.845193, recall 0.471787
---> step 15, loss 1.12476, recall 0.542903
---> step 16, loss 0.932038, recall 0.518124
---> step 17, loss 0.982439, recall 0.529106
---> step 18, loss 0.90561, recall 0.501219
---> step 19, loss 0.678753, recall 0.544872
---> step 20, loss 0.969584, recall 0.493878
---> step 21, loss 1.02145, recall 0.489583
---> step 22, loss 1.06619, recall 0.480101
---> step 23, loss 1.20629, reca

---> step 186, loss 0.591312, recall 0.535044
---> step 187, loss 0.597161, recall 0.521421
---> step 188, loss 0.626864, recall 0.528418
---> step 189, loss 0.504588, recall 0.607823
---> step 190, loss 0.525638, recall 0.56289
---> step 191, loss 0.504776, recall 0.614768
---> step 192, loss 0.722549, recall 0.544393
---> step 193, loss 0.52178, recall 0.548509
---> step 194, loss 0.520213, recall 0.633917
---> step 195, loss 0.488001, recall 0.565705
---> step 196, loss 0.482862, recall 0.59777
---> step 197, loss 0.555968, recall 0.581874
---> step 198, loss 0.514738, recall 0.540752
---> step 199, loss 0.507671, recall 0.576533
---> step 200, loss 0.573279, recall 0.595023

Evaluation:
---> step 200, loss 0.523445, recall 0.580317, acc 0.588199, F1 score 0.584231

---> step 201, loss 0.486596, recall 0.533448
---> step 202, loss 0.454874, recall 0.552316
---> step 203, loss 0.406793, recall 0.719655
---> step 204, loss 0.520232, recall 0.466683
---> step 205, loss 0.516849, recall

---> step 367, loss 0.563745, recall 0.592995
---> step 368, loss 0.423801, recall 0.681287
---> step 369, loss 0.506753, recall 0.595119
---> step 370, loss 0.437106, recall 0.681165
---> step 371, loss 0.462501, recall 0.61677
---> step 372, loss 0.484186, recall 0.640488
---> step 373, loss 0.301861, recall 0.770714
---> step 374, loss 0.48169, recall 0.615878
---> step 375, loss 0.417754, recall 0.70242
---> step 376, loss 0.457851, recall 0.565705
---> step 377, loss 0.426122, recall 0.649038
---> step 378, loss 0.338111, recall 0.753575
---> step 379, loss 0.41392, recall 0.696078
---> step 380, loss 0.437958, recall 0.705223
---> step 381, loss 0.406806, recall 0.653093
---> step 382, loss 0.405818, recall 0.600583
---> step 383, loss 0.443198, recall 0.619819
---> step 384, loss 0.374875, recall 0.654865
---> step 385, loss 0.398566, recall 0.679592
---> step 386, loss 0.492961, recall 0.650877
---> step 387, loss 0.409712, recall 0.683734
---> step 388, loss 0.434823, recall 0

#### 6.3.2 Subtask B: Normal Dictionary with LSTM (Not Split)

In [15]:
senti_analy(B_dict, model_MODE = 1, change_flag = True)

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.589047, recall 0.5
---> step 2, loss 1.76065, recall 0.5
---> step 3, loss 0.575594, recall 0.5
---> step 4, loss 0.69627, recall 0.5
---> step 5, loss 0.822893, recall 0.5
---> step 6, loss 1.28112, recall 0.5
---> step 7, loss 0.769215, recall 0.5
---> step 8, loss 1.02628, recall 0.5
---> step 9, loss 0.624912, recall 0.5
---> step 10, loss 0.544497, recall 0.5
---> step 11, loss 0.675173, recall 0.495146
---> step 12, loss 0.766218, recall 0.483871
---> step 13, loss 0.700097, recall 0.407143
---> step 14, loss 0.576693, recall 0.5
---> step 15, loss 0.523608, recall 0.5
---> step 16, loss 0.465666, recall 0.5
---> step 17, loss 0.550201, recall 0.5
---> step 18, loss 0.825178, recall 0.5
---> step 19, loss 0.764458, recall 0.5
---> step 20, loss 0.609305, recall 0.5
---> step 21, loss 0.513058, recall 0.5
---> step 22, loss 0.5919, recall 0.5
---> step 23, loss 0.545521, recall 0.5
---> step 24, loss 0.621018, recall 0.5
---> step 25, loss 0.615264, recall 0.5


---> step 195, loss 0.382067, recall 0.752039
---> step 196, loss 0.262089, recall 0.828231
---> step 197, loss 0.321035, recall 0.770874
---> step 198, loss 0.356935, recall 0.636111
---> step 199, loss 0.421774, recall 0.691429
---> step 200, loss 0.333968, recall 0.727137

Evaluation:
---> step 200, loss 0.319829, recall 0.738111, acc 0.786859, F1 score 0.761706

---> step 201, loss 0.313902, recall 0.773379
---> step 202, loss 0.303456, recall 0.735294
---> step 203, loss 0.305154, recall 0.829731
---> step 204, loss 0.341773, recall 0.780161
---> step 205, loss 0.301868, recall 0.787241
---> step 206, loss 0.327952, recall 0.792866
---> step 207, loss 0.313738, recall 0.840498
---> step 208, loss 0.430481, recall 0.719113
---> step 209, loss 0.326686, recall 0.808575
---> step 210, loss 0.348261, recall 0.741987
---> step 211, loss 0.326768, recall 0.743124
---> step 212, loss 0.289919, recall 0.772089
---> step 213, loss 0.337141, recall 0.803292
---> step 214, loss 0.28057, reca

---> step 372, loss 0.10068, recall 0.937474
---> step 373, loss 0.218488, recall 0.868027
---> step 374, loss 0.141196, recall 0.877019
---> step 375, loss 0.176609, recall 0.894624
---> step 376, loss 0.160085, recall 0.887762
---> step 377, loss 0.187356, recall 0.893996
---> step 378, loss 0.232914, recall 0.852143
---> step 379, loss 0.190076, recall 0.913078
---> step 380, loss 0.210358, recall 0.884615
---> step 381, loss 0.16061, recall 0.935214
---> step 382, loss 0.191535, recall 0.869388
---> step 383, loss 0.189675, recall 0.884238
---> step 384, loss 0.193517, recall 0.903891
---> step 385, loss 0.151128, recall 0.907051
---> step 386, loss 0.167472, recall 0.866667
---> step 387, loss 0.263008, recall 0.913636
---> step 388, loss 0.137462, recall 0.936149
---> step 389, loss 0.157619, recall 0.927225
---> step 390, loss 0.270153, recall 0.891456
---> step 391, loss 0.162251, recall 0.919845
---> step 392, loss 0.206938, recall 0.87635
---> step 393, loss 0.139957, recall 

#### 6.3.3 Subtask B: GloVe Dictionary with CNN (Not Split)

In [16]:
senti_analy(B_GloVe, model_MODE = 0, change_flag = True)

INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 1.04191, recall 0.528389
---> step 2, loss 0.945997, recall 0.477033
---> step 3, loss 0.730498, recall 0.468518
---> step 4, loss 0.805328, recall 0.514668
---> step 5, loss 0.874524, recall 0.535584
---> step 6, loss 0.868969, recall 0.483786
---> step 7, loss 0.820995, recall 0.51282
---> step 8, loss 0.986941, recall 0.469278
---> step 9, loss 0.798212, recall 0.440142
---> step 10, loss 0.856259, recall 0.456429
---> step 11, loss 1.01701, recall 0.503061
---> step 12, loss 0.865069, recall 0.482644
---> step 13, loss 0.892855, recall 0.538043
---> step 14, loss 0.94258, recall 0.545682
---> step 15, loss 0.754888, recall 0.442913
---> step 16, loss 0.569122, recall 0.589744
---> step 17, loss 1.1439, recall 0.481818
---> step 18, loss 0.759242, recall 0.558833
---> step 19, loss 0.619503, recall 0.529818
---> step 20, loss 0.840257, recall 0.483409
---> step 21, loss 0.730075, recall 0.59736
---> step 22, loss 0.74812, recall 0.514286
---> step 23, loss 0.701508

---> step 181, loss 0.46698, recall 0.543836
---> step 182, loss 0.610789, recall 0.585088
---> step 183, loss 0.572536, recall 0.6093
---> step 184, loss 0.479089, recall 0.566019
---> step 185, loss 0.472666, recall 0.526587
---> step 186, loss 0.497824, recall 0.590387
---> step 187, loss 0.476171, recall 0.595437
---> step 188, loss 0.602615, recall 0.561594
---> step 189, loss 0.524838, recall 0.660766
---> step 190, loss 0.503077, recall 0.60169
---> step 191, loss 0.567948, recall 0.472857
---> step 192, loss 0.51141, recall 0.570224
---> step 193, loss 0.495951, recall 0.620714
---> step 194, loss 0.61874, recall 0.535484
---> step 195, loss 0.515771, recall 0.65625
---> step 196, loss 0.568534, recall 0.592476
---> step 197, loss 0.528549, recall 0.570185
---> step 198, loss 0.469624, recall 0.65625
---> step 199, loss 0.501491, recall 0.597857
---> step 200, loss 0.514249, recall 0.594896

Evaluation:
---> step 200, loss 0.529005, recall 0.629187, acc 0.673671, F1 score 0.650

---> step 357, loss 0.388999, recall 0.659627
---> step 358, loss 0.436805, recall 0.644556
---> step 359, loss 0.388918, recall 0.717143
---> step 360, loss 0.404915, recall 0.579592
---> step 361, loss 0.414678, recall 0.651646
---> step 362, loss 0.387061, recall 0.656136
---> step 363, loss 0.291508, recall 0.740741
---> step 364, loss 0.360847, recall 0.640893
---> step 365, loss 0.396206, recall 0.632857
---> step 366, loss 0.368642, recall 0.716667
---> step 367, loss 0.350935, recall 0.659352
---> step 368, loss 0.393162, recall 0.641156
---> step 369, loss 0.37904, recall 0.648033
---> step 370, loss 0.392891, recall 0.69747
---> step 371, loss 0.366509, recall 0.73718
---> step 372, loss 0.277204, recall 0.814642
---> step 373, loss 0.431236, recall 0.600962
---> step 374, loss 0.358201, recall 0.628205
---> step 375, loss 0.425246, recall 0.635714
---> step 376, loss 0.336939, recall 0.693803
---> step 377, loss 0.408369, recall 0.592285
---> step 378, loss 0.455558, recall 

#### 6.3.4 Subtask B: GloVe Dictionary with LSTM (Not Split)

In [17]:
senti_analy(B_GloVe, model_MODE = 1, change_flag = True)

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.608682, recall 0.5
---> step 2, loss 1.18133, recall 0.5
---> step 3, loss 0.524403, recall 0.5
---> step 4, loss 0.777363, recall 0.5
---> step 5, loss 0.722921, recall 0.5
---> step 6, loss 0.859168, recall 0.5
---> step 7, loss 0.759529, recall 0.5
---> step 8, loss 0.590243, recall 0.5
---> step 9, loss 0.515678, recall 0.5
---> step 10, loss 0.625424, recall 0.5
---> step 11, loss 0.688625, recall 0.492517
---> step 12, loss 0.657234, recall 0.5
---> step 13, loss 0.590764, recall 0.5
---> step 14, loss 0.52075, recall 0.5
---> step 15, loss 0.512132, recall 0.5
---> step 16, loss 0.620935, recall 0.5
---> step 17, loss 0.501431, recall 0.5
---> step 18, loss 0.752466, recall 0.5
---> step 19, loss 0.69565, recall 0.5
---> step 20, loss 0.549885, recall 0.5
---> step 21, loss 0.534113, recall 0.5
---> step 22, loss 0.492559, recall 0.5
---> step 23, loss 0.548528, recall 0.5
---> step 24, loss 0.527788, recall 0.5
---> step 25, loss 0.549163, recall 0.5
---> st

---> step 194, loss 0.329496, recall 0.7
---> step 195, loss 0.411713, recall 0.739583
---> step 196, loss 0.39421, recall 0.74898
---> step 197, loss 0.310571, recall 0.837321
---> step 198, loss 0.322151, recall 0.825816
---> step 199, loss 0.416837, recall 0.705714
---> step 200, loss 0.283072, recall 0.83

Evaluation:
---> step 200, loss 0.292261, recall 0.781429, acc 0.797134, F1 score 0.789203

---> step 201, loss 0.452205, recall 0.644898
---> step 202, loss 0.445267, recall 0.651042
---> step 203, loss 0.296669, recall 0.854235
---> step 204, loss 0.355283, recall 0.811463
---> step 205, loss 0.319364, recall 0.807841
---> step 206, loss 0.324506, recall 0.696039
---> step 207, loss 0.345118, recall 0.754525
---> step 208, loss 0.341694, recall 0.772283
---> step 209, loss 0.33585, recall 0.797129
---> step 210, loss 0.235538, recall 0.855204
---> step 211, loss 0.22559, recall 0.835334
---> step 212, loss 0.225847, recall 0.862733
---> step 213, loss 0.256685, recall 0.875782


---> step 370, loss 0.139715, recall 0.903692
---> step 371, loss 0.15804, recall 0.902244
---> step 372, loss 0.243373, recall 0.871591
---> step 373, loss 0.124345, recall 0.963542
---> step 374, loss 0.154225, recall 0.926429
---> step 375, loss 0.107107, recall 0.968027
---> step 376, loss 0.213433, recall 0.916667
---> step 377, loss 0.115563, recall 0.951766
---> step 378, loss 0.166496, recall 0.9199
---> step 379, loss 0.123145, recall 0.955146
---> step 380, loss 0.0864638, recall 0.972973
---> step 381, loss 0.215994, recall 0.932292
---> step 382, loss 0.280273, recall 0.857018
---> step 383, loss 0.0722564, recall 0.95176
---> step 384, loss 0.151415, recall 0.90352
---> step 385, loss 0.231897, recall 0.885417
---> step 386, loss 0.18906, recall 0.880583
---> step 387, loss 0.156383, recall 0.890874
---> step 388, loss 0.281527, recall 0.797283
---> step 389, loss 0.259057, recall 0.885417
---> step 390, loss 0.182162, recall 0.900583
---> step 391, loss 0.200898, recall 0

### 6.4 Subtask B: Split by Topic

#### 6.4.1 Subtask B: Normal Dictionary with CNN (Split)

In [18]:
senti_analy(B_dict, model_MODE = 0, change_flag = False)

--->Topic 0: Now training for topic 'amy schumer'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary na

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.626526, recall 0.694444
---> step 2, loss 1.21058, recall 0.433333
---> step 3, loss 0.965801, recall 0.494444
---> step 4, loss 0.813332, recall 0.631482
---> step 5, loss 0.868671, recall 0.594444
---> step 6, loss 1.02282, recall 0.451852
---> step 7, loss 0.763112, recall 0.601852
---> step 8, loss 0.764859, recall 0.601852
---> step 9, loss 0.844739, recall 0.612963
---> step 10, loss 0.752889, recall 0.651852

Training Result:
---> step 10, loss 0.862439, recall 0.557407, acc 0.56681, F1 score 0.56207
--->Topic 1: Now training for topic 'ant-man'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpoo

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.765976, recall 0.592742
---> step 2, loss 0.895814, recall 0.866667
---> step 3, loss 0.560167, recall 0.494309
---> step 4, loss 0.205671, recall 0.4375
---> step 5, loss 0.437514, recall 0.547154
---> step 6, loss 0.202669, recall 0.4375
---> step 7, loss 0.405127, recall 0.45935
---> step 8, loss 0.355816, recall 0.4375
---> step 9, loss 0.250811, recall 0.579675
---> step 10, loss 0.0116974, recall 0.5
---> step 11, loss 0.243219, recall 0.487805
---> step 12, loss 0.0399219, recall 0.5
---> step 13, loss 0.321362, recall 0.487805
---> step 14, loss 0.0290086, recall 0.5
---> step 15, loss 0.179383, recall 0.5
---> step 16, loss 0.721003, recall 0.5
---> step 17, loss 0.225886, recall 0.5
---> step 18, loss 0.454788, recall 0.5
---> step 19, loss 0.251872, recall 0.495935
---> step 20, loss 0.0192534, recall 0.5

Training Result:
---> step 20, loss 0.0165098, recall 0.5, acc 0.5, F1 score 0.5
--->Topic 2: Now training for topic 'bad blood'
INFO:tensorflow:Summar

...

---> step 1, loss 0.542215, recall 0.877551
---> step 2, loss 0.441573, recall 0.887755
---> step 3, loss 0.367821, recall 0.438776
---> step 4, loss 0.290313, recall 0.428571
---> step 5, loss 0.337106, recall 0.428571
---> step 6, loss 0.365889, recall 0.438776
---> step 7, loss 0.204092, recall 0.489796
---> step 8, loss 0.205076, recall 0.479592
---> step 9, loss 0.147513, recall 0.479592
---> step 10, loss 0.171577, recall 0.489796

Training Result:
---> step 10, loss 0.118404, recall 0.489796, acc 0.489796, F1 score 0.489796
--->Topic 98: Now training for topic 'zac brown band'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 1.63645, recall 0.517157
---> step 2, loss 1.55768, recall 0.35049
---> step 3, loss 1.0708, recall 0.42402
---> step 4, loss 1.02527, recall 0.634804
---> step 5, loss 0.846564, recall 0.490196
---> step 6, loss 0.625508, recall 0.345588
---> step 7, loss 0.531678, recall 0.534314
---> step 8, loss 0.369771, recall 0.745098
---> step 9, loss 0.354848, recall 0.622549
---> step 10, loss 0.364367, recall 0.455882

Training Result:
---> step 10, loss 0.289894, recall 0.477941, acc 0.477941, F1 score 0.477941
--->Topic 99: Now training for topic 'zayn'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 4.14232, recall 0.523077
---> step 2, loss 4.0933, recall 0.451648
---> step 3, loss 3.69188, recall 0.546154
---> step 4, loss 3.10248, recall 0.538462
---> step 5, loss 2.76952, recall 0.513187
---> step 6, loss 2.59179, recall 0.520879
---> step 7, loss 2.03396, recall 0.510989
---> step 8, loss 1.90385, recall 0.454945
---> step 9, loss 1.51625, recall 0.43956
---> step 10, loss 1.04349, recall 0.674725

Training Result:
---> step 10, loss 1.29007, recall 0.493407, acc 0.497619, F1 score 0.495504
For task B, recall is 0.561905, accuracy is 0.566823, F1 score is 0.562999


#### 6.4.2 Subtask B: Normal Dictionary with LSTM (Split)

In [19]:
senti_analy(B_dict, model_MODE = 1, change_flag = False)

--->Topic 0: Now training for topic 'amy schumer'


D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.724501, recall 0.5
---> step 2, loss 2.28198, recall 0.5
---> step 3, loss 0.752361, recall 0.437037
---> step 4, loss 0.794436, recall 0.5
---> step 5, loss 1.11874, recall 0.5
---> step 6, loss 1.19062, recall 0.5
---> step 7, loss 1.08345, recall 0.5
---> step 8, loss 0.874676, recall 0.5
---> step 9, loss 0.654497, recall 0.5
---> step 10, loss 0.580692, recall 0.5

Training Result:
---> step 10, loss 0.738486, recall 0.5, acc 0.135135, F1 score 0.212766
--->Topic 1: Now training for topic 'ant-man'


...

---> step 1, loss 2.10335, recall 0.5
---> step 2, loss 0.183268, recall 0.5
---> step 3, loss 0.241269, recall 0.5
---> step 4, loss 0.325676, recall 0.5
---> step 5, loss 0.383852, recall 0.5
---> step 6, loss 0.42123, recall 0.5
---> step 7, loss 0.442804, recall 0.5
---> step 8, loss 0.452273, recall 0.5
---> step 9, loss 0.452439, recall 0.5
---> step 10, loss 0.44546, recall 0.5

Training Result:
---> step 10, loss 0.433022, recall 0.5, acc 0.478873, F1 score 0.489209
--->Topic 99: Now training for topic 'zayn'


D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.514677, recall 0.5
---> step 2, loss 0.333613, recall 0.5
---> step 3, loss 0.346902, recall 0.5
---> step 4, loss 0.321167, recall 0.5
---> step 5, loss 0.346518, recall 0.5
---> step 6, loss 0.335569, recall 0.5
---> step 7, loss 0.313426, recall 0.5
---> step 8, loss 0.321554, recall 0.5
---> step 9, loss 0.330876, recall 0.5
---> step 10, loss 0.313435, recall 0.5

Training Result:
---> step 10, loss 0.311453, recall 0.5, acc 0.451389, F1 score 0.474453
For task B, recall is 0.501166, accuracy is 0.4079, F1 score is 0.442876


#### 6.4.3 Subtask B: GloVe Dictionary with CNN (Split)

In [20]:
senti_analy(B_GloVe, model_MODE = 0, change_flag = False)

--->Topic 0: Now training for topic 'amy schumer'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary na

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 1.2364, recall 0.444444
---> step 2, loss 0.804554, recall 0.494444
---> step 3, loss 0.756665, recall 0.507407
---> step 4, loss 0.773209, recall 0.657407
---> step 5, loss 1.16607, recall 0.57037
---> step 6, loss 0.885749, recall 0.488889
---> step 7, loss 0.572064, recall 0.644444
---> step 8, loss 0.556131, recall 0.657407
---> step 9, loss 0.428942, recall 0.675926
---> step 10, loss 0.706374, recall 0.525926

Training Result:
---> step 10, loss 0.979688, recall 0.52037, acc 0.52037, F1 score 0.52037
--->Topic 1: Now training for topic 'ant-man'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 0.67413, recall 0.358871
---> step 2, loss 0.52375, recall 0.4
---> step 3, loss 0.344099, recall 0.45935
---> step 4, loss 0.0360899, recall 0.5
---> step 5, loss 0.285908, recall 0.655285
---> step 6, loss 0.177599, recall 0.46875
---> step 7, loss 0.216256, recall 0.483871
---> step 8, loss 0.189985, recall 0.966667
---> step 9, loss 0.324159, recall 0.47561
---> step 10, loss 0.0404312, recall 0.5
---> step 11, loss 0.259667, recall 0.48374
---> step 12, loss 0.032222, recall 0.5
---> step 13, loss 0.20009, recall 0.495935
---> step 14, loss 0.00143861, recall 0.5
---> step 15, loss 0.183912, recall 0.5
---> step 16, loss 0.273431, recall 0.5
---> step 17, loss 0.249358, recall 0.5
---> step 18, loss 0.0602987, recall 0.46875
---> step 19, loss 0.197449, recall 0.5
---> step 20, loss 0.0137396, recall 0.5

Training Result:
---> step 20, loss 0.00318102, recall 0.5, acc 0.5, F1 score 0.5
--->Topic 2: Now training for topic 'bad blood'
INFO:tensorflow:Summary name E

...

---> step 1, loss 2.69741, recall 0.0714286
---> step 2, loss 1.83746, recall 0.683674
---> step 3, loss 1.36848, recall 0.693878
---> step 4, loss 1.34139, recall 0.255102
---> step 5, loss 1.07111, recall 0.27551
---> step 6, loss 1.04348, recall 0.295918
---> step 7, loss 0.61632, recall 0.377551
---> step 8, loss 0.540195, recall 0.846939
---> step 9, loss 0.516565, recall 0.428571
---> step 10, loss 0.547704, recall 0.418367

Training Result:
---> step 10, loss 0.24628, recall 0.438776, acc 0.488636, F1 score 0.462366
--->Topic 98: Now training for topic 'zac brown band'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-ma

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 1.59246, recall 0.213235
---> step 2, loss 1.16952, recall 0.612745
---> step 3, loss 0.904888, recall 0.294118
---> step 4, loss 0.650392, recall 0.54902
---> step 5, loss 0.636135, recall 0.693627
---> step 6, loss 0.468181, recall 0.411765
---> step 7, loss 0.421706, recall 0.448529
---> step 8, loss 0.395959, recall 0.441176
---> step 9, loss 0.260358, recall 0.485294
---> step 10, loss 0.272984, recall 0.485294

Training Result:
---> step 10, loss 0.34351, recall 0.629902, acc 0.567949, F1 score 0.597323
--->Topic 99: Now training for topic 'zayn'
INFO:tensorflow:Summary name Embedding/W:0/grad/hist is illegal; using Embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name Embedding/W:0/grad/sparsity is illegal; using Embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 1.14948, recall 0.412088
---> step 2, loss 0.670447, recall 0.527473
---> step 3, loss 0.959234, recall 0.458242
---> step 4, loss 0.600498, recall 0.407692
---> step 5, loss 0.745667, recall 0.4
---> step 6, loss 0.509888, recall 0.50989
---> step 7, loss 0.471177, recall 0.668132
---> step 8, loss 0.433012, recall 0.517582
---> step 9, loss 0.378718, recall 0.492308
---> step 10, loss 0.250558, recall 0.476923

Training Result:
---> step 10, loss 0.537361, recall 0.492308, acc 0.450704, F1 score 0.470588
For task B, recall is 0.561837, accuracy is 0.567949, F1 score is 0.56381


#### 6.4.4 Subtask B: GloVe Dictionary with LSTM (Split)

In [21]:
senti_analy(B_GloVe, model_MODE = 1, change_flag = False)

--->Topic 0: Now training for topic 'amy schumer'


D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 1.04033, recall 0.5
---> step 2, loss 0.673772, recall 0.5
---> step 3, loss 0.703053, recall 0.266667
---> step 4, loss 0.588978, recall 0.5
---> step 5, loss 0.639057, recall 0.5
---> step 6, loss 0.651109, recall 0.5
---> step 7, loss 0.607914, recall 0.5
---> step 8, loss 0.579579, recall 0.5
---> step 9, loss 0.603922, recall 0.5
---> step 10, loss 0.618066, recall 0.5

Training Result:
---> step 10, loss 0.592092, recall 0.5, acc 0.364865, F1 score 0.421875
--->Topic 1: Now training for topic 'ant-man'


...

---> step 1, loss 0.240761, recall 0.5
---> step 2, loss 0.245254, recall 0.5
---> step 3, loss 0.298995, recall 0.5
---> step 4, loss 0.297967, recall 0.5
---> step 5, loss 0.267514, recall 0.5
---> step 6, loss 0.223114, recall 0.5
---> step 7, loss 0.181752, recall 0.5
---> step 8, loss 0.184067, recall 0.5
---> step 9, loss 0.227474, recall 0.5
---> step 10, loss 0.198896, recall 0.5

Training Result:
---> step 10, loss 0.174425, recall 0.5, acc 0.478873, F1 score 0.489209
--->Topic 99: Now training for topic 'zayn'


D:\Anaconda\envs\AMLS_Lab\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/hist is illegal; using rnn/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0/grad/sparsity is illegal; using rnn/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/hist is illegal; using rnn/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0/grad/sparsity is illegal; using rnn/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable:0/grad/hist is illegal; using Variable_0/grad/hist instead.
INFO:tensorflow:Summary name Variable:0/grad/sparsity is illegal; using Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name Variable_1:0/grad/hist is illegal; using Variable_1_0/g

D:\Anaconda\envs\AMLS_Lab\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


---> step 1, loss 2.08009, recall 0.5
---> step 2, loss 0.357421, recall 0.5
---> step 3, loss 0.455265, recall 0.5
---> step 4, loss 0.618952, recall 0.5
---> step 5, loss 0.71398, recall 0.5
---> step 6, loss 0.754259, recall 0.5
---> step 7, loss 0.755168, recall 0.5
---> step 8, loss 0.728116, recall 0.5
---> step 9, loss 0.681296, recall 0.5
---> step 10, loss 0.620671, recall 0.5

Training Result:
---> step 10, loss 0.550895, recall 0.5, acc 0.451389, F1 score 0.474453
For task B, recall is 0.501066, accuracy is 0.405035, F1 score is 0.440078
